# Foundations of Artificial Intelligence and Machine Learning
## A Program by IIIT-H and TalentSprint

#### To be done in the Lab

The objective of this experiment is to perform sentimental analysis.

In this experiment we will be using twitter dataset as training data and crawled realtime tweets for testing. 

The Ground truth is 1 for positive tweet and 0 for negative tweet.

Few examples of positive and negative tweets are:

**Few Positive Tweets: **
1.  @Msdebramaye I heard about that contest! Congrats girl!!
2. UNC!!! NCAA Champs!! Franklin St.: I WAS THERE!! WILD AND CRAZY!!!!!! Nothing like it...EVER http://tinyurl.com/49955t3

**Few Negative Tweets:**
1. no more taking Irish car bombs with strange Australian women who can drink like rockstars...my head hurts.
2. Just had some bloodwork done. My arm hurts

### Data Source

https://www.kaggle.com/c/twitter-sentiment-analysis2/data


## Exercise 1: (2 marks)

The first exercise is cleaning the tweets.
Perform preprocessing as required.

Complete the functon : preprocess_tweets 

Input or Arguement to the function : tweet as a string 

Return value: processed tweet as string 

Hint: Use regular expressions
* convert the all the cases into lower case
  + look at lower()
* Replace any urls with the word "URL"
  + Hint : 
      - re.sub('((www\.[^\s]+)|(https?://[^\s]+))','URL',"Tweet") (re is python regular expression package)
* convert the username to "AT_USER", consider any word that starts with @ as user name
  + Hint : 
      - re.sub('@[^\s]+','AT_USER',"Tweet")
* Remove multiple whitespaces with a single white space
  + Hint :
      - re.sub('[\s]+', ' ', tweet)
* Replace hashtag words (#word) with just the words (word)
  + Hint : 
      - re.sub(r'#([^\s]+)', r'\1', "tweet")
      
* TEST CASE :
    + given the tweet "@V_DEL_ROSSI: Me         #dragging myself to the gym https://t.co/cOjM0mBVeY"
    + output should be "AT_USER me dragging myself to the gym URL"

In [1]:
import re
import gensim
# Operating System
import os
# Regular Expression
import re
# nltk packages
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.porter import *
from nltk.stem.snowball import SnowballStemmer
# Basic Packages
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
# PCA Package
from sklearn.decomposition import PCA

In [5]:
def preprocess_tweets(tweet):
    tweet=tweet.lower()
    tweet = re.sub('((www.[^\s]+)|(https?://[^\s]+))','URL',tweet)
    tweet=re.sub('@[^\s]+','AT_USER',tweet)
    tweet=re.sub('[\s]+', ' ', tweet)
    tweet=re.sub(r'#([^\s]+)', r'\1', tweet)
    tweet=re.sub('&lt;','',tweet)#xml tag
    tweet=re.sub('&amp;','',tweet)
    tweet=re.sub(':o','smile',tweet) #replace smileys with good or bad emotion
    tweet=re.sub(':-\|','speechless',tweet)
    tweet=re.sub('[.!,-]+', '', tweet)
    return tweet

print(preprocess_tweets("@V_DEL_ROSSI: Me #dragging myself to the gym https://t.co/cOjM0mBVeY"))

AT_USER me dragging myself to the gym URL


In [9]:
import pandas as pd
data = pd.read_csv('twitter_train.csv',encoding='latin-1',converters={'SentimentText':preprocess_tweets})
pw=pd.read_csv('positive-words.txt').values
nw=pd.read_csv('negative-words.txt').values

stopWords = pd.read_csv('stopwords.txt').values
stemmer = SnowballStemmer("english")
vocabulary = set([])

data.head()

,ItemID,Sentiment,SentimentText
0,1,0,is so sad for my apl friend
1,2,0,i missed the new moon trailer
2,3,1,omg its already 7:30 smile
3,4,0,omgaga im sooo im gunna cry i've been at thi...
4,5,0,i think mi bf is cheating on me t_t


## Exercise 2: (3 marks)

Tokenize the processed tweets to make a tweet into a list of words and make sure that no punctuations are returned. so that it can be used in the next steps to represent the tweet as a feature vector. Remove the Stops words, if necessary

Complete the functon : word_tokenizer 

Input or Arguement to the function : processed tweet

Return value: list of words without any punctuations

TEST CASE :

Given an input :
    "Neither Man, nor machine can replace its creator. really?."
    
Result : 
    ['neither', 'man', 'nor', 'machine', 'replace', 'creator', 'hahaha']

In [11]:
def word_tokenizer(tweet):
    words = re.findall(r'(\b[A-Za-z][a-z]{2,15}\b)', tweet)
    words = [ stemmer.stem(word.lower()) for word in words if not word.lower() in stopWords]
    return words

In [10]:
for i in range(data.values.shape[0]):
    words= word_tokenizer(data.values[i][2])
    for word in words:
          vocabulary.add(word)
vocabularyList=list(vocabulary)

NameError: name 'word_tokenizer' is not defined

In [ ]:
# add postive words count, negative wourd count, neutral, and tweet (label)
voclength=len(vocabulary)
fvlength=voclength+4

## Exercise 3: (5 marks)

Using the list of words from the above the step, 
* represent the tweet as a feature vector using bag of words/Word2vec

Hint : counts of postive/negative/neutral words as three features can also be used

In [ ]:
def getfeaturevector(tokenized_tweet,sentiment):
    feature_vector=np.zeros(fvlength)
    for word in tokenized_tweet: 
        if vocabularyList.index(word) !=-1:
            feature_vector[vocabularyList.index(word)]+=1
        flag=0
        for i in range(len(pw)):
            if word == pw[i][0]:
                feature_vector[voclength]+=1
                flag=1
        for i in range(len(nw)):
            if word == nw[i][0]:
                feature_vector[voclength+1]+=1
                flag=-1
        if flag==0:
            feature_vector[voclength+2]+=1
        feature_vector[voclength+3]=sentiment
    return feature_vector

In [ ]:
finaldata=[]
for i in range(data.values.shape[0]):
    finaldata.append(getfeaturevector(word_tokenizer(data.values[i][2]),data.values[i][1]))

## Exercise 4: (Marks : 5 ) 


Load the given training data and use the above functions you created to process, to tokenise and to get feature vector.

Considering the feature vector as input to the classifier, Train a classifier to classify the sentiment of the tweet correctly.

Divide the training data into two sets, to validate your classifier

In [ ]:
print
# Your Code Hereimport random
train = []
test = []

def randomize():
    TRAIN_TEST_RATIO = 0.8
    data = finaldata
    #print(data)
    for one in data:
        #print(one)
        if random.random() < TRAIN_TEST_RATIO:
            train.append(one)
        else:
            test.append(one)
    print (test[:10])
    
randomize()

In [ ]:
import math
import collections
def dist(a, b):
    sqSum = 0
    for i in np.arange( len(a)):
        sqSum += (a[i] - b[i]) ** 2
    return math.sqrt(sqSum)
# ------------------------------------------------ #
# We are assuming that the label is the last field #
# If not, munge the data to make it so!            #
# ------------------------------------------------ #
def kNN(k, train, given):
    distances = []
    for t in train:
        distance=(dist(t[:-1], given), str(int(t[-1])))
        #print(distance)
        distances.append(distance)
    distances.sort()
    return distances[:k]

def kNN_classify(k, train, given):
    tally = collections.Counter()
    #print(tally)
    for nn in kNN(k, train, given):
        tally.update(nn[-1])
    return tally.most_common(1)[0]

In [ ]:
def KNN_accuracy(Kvalue):
    num = 0
    print("calculating accuracy")
    for t in test:
        prediction = kNN_classify(Kvalue, train, t)[0]
        #print("prediction, label is", prediction, t[134])
        if (int(prediction) == int(t[index])):
            num += 1
        #print(kNN_classify(5, train, t)[0], t[-1])
    #print(num/len(test))
    return (num/len(test))

In [ ]:
KNN_accuracy(3)

## Exercise 5: (Marks : 5)

#### Twitter crawling using tweepy

Use tweepy to get the tweets on real time, which is used as test data for the classifier.

## Requirements: 

Twitter account

Create a twitter account if you don't have one by going to the link given below:

https://twitter.com/i/flow/signup

## Installation

Tweepy: tweepy is the python client for the official Twitter API.
Install it using following pip command:

In [1]:
!pip install tweepy

The tweets need to be gathered so as to perform Sentiment analysis on those tweets. They can be fetched from Twitter using the Twitter API. 

In order to fetch tweets through Twitter API, one needs to register an App through their twitter account. Follow these steps for the same:
<ul>
<li>Open the link given below to create a App through the twitter account.
    https://apps.twitter.com
<li>click the button: ‘Create New App’
<li>Fill the application details. You can leave the callback url field empty.
<li>Once the app is created, you will be redirected to the app page.
<li>Open the ‘Keys and Access Tokens’ tab.
<li>Copy ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’.
</ul>

In [6]:
#Replace with your ‘Consumer Key’, ‘Consumer Secret’, ‘Access token’ and ‘Access Token Secret’ below. 

consumer_key = "sjqCktpagvR4NjTsUR3DcA7pp"
consumer_secret = "dqxiWPhqt7n7uR0tGqA18bPCRsXxtEPVNg3HmOtfmzjjjMEXwa"
access_token =  "968060979681681408-kTK6GHKeBRr3oyEalq3NwQKtC9Vn4fS"
access_secret = "pICNG5EIzlBwf0vmSLCitRmtloKr1J1Hmd6w8JixWDmKU"

Run the code below to authenticate your application.

In [7]:
import tweepy

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

api = tweepy.API(auth)    

## Tweepy Cursor

The below code gives the search results from twitter for the search string passed to the keyword arguement "q" in the tweepy.Cursor. The number passed to the items method of tweepy.Cursor indicates that it gives 100 such tweets, if available

In [8]:
for i in tweepy.Cursor(api.search, q='bankofamerica', lang = 'en', full_text=True).items(100):
    print(i._json['text'])
    #print(myStreamListener.processTweet(i._json['text']), end='\n\n\n')

RT @Lilia_Ciciolla: Another great Friday night in #FortLauderdale supporting @HendersonHlth at @BankofAmerica #StarlightMusicals in partner…
RT @BankofAmerica: These brothers take teamwork to a new level as @SpecialOlympics athletes. Wish them luck at the @2018USAGames. https://t…
RT @BankofAmerica: .@SpecialOlympics has changed the lives of two of our employees who are competing at the @2018USAgames #PickUpHope https…
RT @BankofAmerica: .@SpecialOlympics has changed the lives of two of our employees who are competing at the @2018USAgames #PickUpHope https…
Bank of America Museums On Us Program https://t.co/NZ3PEw0Go3
6 U.S. banks finance ICE detention centers: @BankofAmerica @jpmorgan @BNPParibas @SunTrust @usbank &amp; @WellsFargo pr… https://t.co/x0wNYGDyEJ
Happy Pride! Today I get to walk in Pride as the majority. To me that is so powerful &amp; I know together as a communi… https://t.co/SCtwrq1Yvy
RT @Triggered1776: Sunoco has fired #PoolPatrolPeter for racial profiling. Can we ge

Good on you, @BankofAmerica

Thank you for taking care of a man who gave so much to take care of us all

This is th… https://t.co/C5GeCCGoRb
@CamWohl @BankofAmerica Damn dude, almost two days to respond? So which bank did you switch to after you closed your @BankofAmerica account?
RT @BankofAmerica: Proud to support Erin on her once-in-a-lifetime journey to the @SpecialOlympics @2018USAGames in Seattle. #PickUpHope fo…
RT @CamWohl: Wow @BankofAmerica the silent treatment?! Unreal.
RT @CamWohl: @BankofAmerica NOT COOL, BOA. NOT COOL AT ALL. @TDAmeritrade thanks for the great customer service after BOA stole my plastic.…
From @SpokaneCity to the Puget Sound, so many great opportunities to enjoy the arts in Washington State! Thanks,… https://t.co/sZRxGTN3dq
@ZelleSupport If I have been using @Zelle with my PNC account, but now want to switch it to my @BankofAmerica account.  How do I do that?
If I have been using @Zelle with my PNC account, but now want to switch it to my @BankofAmerica a

Also apply the preprocessing steps and obtain the feature vectors for the crawled twitter data.
Classify the crawled tweets by passing its feature vector to the trained classifier.

In [ ]:
##Your code here